In [1]:
import numpy as np
from numpy.linalg import norm
from scipy.spatial import ConvexHull

In [2]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import MeshcatVisualizerParams, Role, GeometrySet, CollisionFilterDeclaration
from pydrake.geometry.optimization import CspaceFreePolytope, SeparatingPlaneOrder
from pydrake.multibody.rational import RationalForwardKinematics
from ciris_plant_visualizer import CIrisPlantVisualizer
import numpy as np

from pydrake.all import (HPolyhedron, AngleAxis,
                         VPolytope, Sphere, Ellipsoid, InverseKinematics,
                         Hyperellipsoid, Simulator, Box)
import mcubes

import visualization_utils as viz_utils

import pydrake.symbolic as sym
from pydrake.all import  TriangleSurfaceMesh, Rgba, SurfaceTriangle, Sphere
from scipy.linalg import null_space
import time

In [3]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

In [4]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from pydrake.all import Rgba
from ciris_plant_visualizer import CIrisPlantVisualizer  # Assuming this is set up properly
from visualization_utils import plot_polytope
from pydrake.systems.framework import DiagramBuilder

In [5]:
import ipywidgets as widgets
from IPython.display import display

In [7]:
# meshcat = StartMeshcat()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the robot
gripper = parser.AddModels(file_name="my_sdfs/wsg_3dof.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0, 0, 0]))

p_GgraspO = [0, 0, .065]
R_GgraspO = RotationMatrix.MakeXRotation(-np.pi / 2)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_wsg", gripper),
    RigidTransform(R_GgraspO, p_GgraspO))

plant.Finalize()

# Release collision constraints
inspector = scene_graph.model_inspector()


# Add visualization
meshcat_params = MeshcatVisualizerParams()
meshcat_params.role = Role.kIllustration

# TCspace
Rat_FK = RationalForwardKinematics(plant)

q_star = np.zeros(plant.num_positions())

# Create the TCspaceFreePolytopes
cspace_free_polytope = CspaceFreePolytope(
    plant, 
    scene_graph,
    SeparatingPlaneOrder.kAffine,
    q_star)

print(plant.num_positions())

visualizer = CIrisPlantVisualizer(
    plant,
    builder,
    scene_graph,
    cspace_free_polytope,
    viz_role=Role.kIllustration,
    allow_plus_3dof=True
)

plant_context = visualizer.plant_context
diagram = visualizer.task_space_diagram
diagram_context = visualizer.task_space_diagram_context

# Set initial configuration
plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.025
)
plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
    plant_context, 0.025
)
diagram.ForcedPublish(diagram_context)

# visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
# visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(np.pi/2), np.array([0,0,1])))

INFO:drake:Meshcat listening for connections at http://localhost:7002
INFO:drake:Meshcat listening for connections at http://localhost:7003


4
Visualisations won't work. Can't visualize the TC-Space of plants with more than 3-DOF


In [8]:
x_init = np.array([-3.14, 1.57, -0.054])
x_goal = np.array([3.14, -1.57, -0.054])

# Define the 8 corner points of the convex hull
x_bounds = [-np.pi, np.pi]
y_bounds = [-np.pi/2, np.pi/2]
z_bounds = [-0.055, -0.025]

lower_joint_limits = np.array([x_bounds[0], y_bounds[0], z_bounds[0]])
upper_joint_limits = np.array([x_bounds[1], y_bounds[1], z_bounds[1]])

z_bounds_grasp = [-0.025, -0.024]

# Generate all corner points
placement_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds])
grasp_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds_grasp])

# Compute the convex hull
placement_hull = ConvexHull(placement_points)
grasp_hull = ConvexHull(grasp_points)

# Get the A and b representation of the half-spaces
def get_halfspaces(hull):
    A = hull.equations[:, :-1]  # Coefficients for x, y, z
    b = -hull.equations[:, -1]  # Right-hand side values
    return A, b

A_p, b_p = get_halfspaces(placement_hull)
A_g, b_g = get_halfspaces(grasp_hull)
assert((A_p @ x_init <= b_p).all())
assert((A_p @ x_goal <= b_p).all())

test_point = np.array([-3.14, 1.57, -0.025])

print("Test point:", test_point)
# Check if the test point is inside the convex hull
is_inside = np.all(A_p @ test_point <= b_p)
print("Is the test point inside the convex placement hull?", is_inside)
is_grasp = np.all(A_g @ test_point <= b_g)
print("Is the test point inside the convex grasp hull?", is_grasp)


Test point: [-3.14   1.57  -0.025]
Is the test point inside the convex placement hull? True
Is the test point inside the convex grasp hull? True


In [9]:
import quadprog

def quadprog_solve_qp(H, l, G=None, h=None, C=None, d=None, verbose=False):
    """
    min (1/2)x' H x + l' x
    subject to  G x <= h
    subject to  C x  = d
    """
    qp_G = 0.5 * (H + H.T)  # make sure P is symmetric
    qp_a = -l
    qp_C = None
    qp_b = None
    meq = 0
    if C is not None: # if equality constraint
        if G is not None: # if both equality and inequality constraints
            qp_C = -np.vstack([C, G]).T
            qp_b = -np.hstack([d, h])
        else: # if only equality constraints
            qp_C = -C.transpose()
            qp_b = -d
        meq = C.shape[0]
    elif G is not None:  # no equality constraint, only inequality constraint
        qp_C = -G.T
        qp_b = -h
    res = quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)
    if verbose:
        return res
    return res[0]

"""
Find [x0, x1g, x2r, x3g, x4r, x5], where g is grasp and r is release
minimise sum
"""


# Define the cost function 1/2 x^T H x + l^T x = ||Cx - c||^2
def to_quadratic_form(C, c):
    return np.dot(C.T, C), -np.dot(C.T, c).flatten()



def solve_for_n_graps(n_grasps, x_init, x_goal, A_p, b_p, A_g, b_g, lower_joint_limits, upper_joint_limits):
    n_points = 2*n_grasps + 2
    n_cols = 3 * n_points


    # Define the cost function 1/2 x^T H x + l^T x = ||Cx - c||^2
    n_rows_cost = n_cols - 3
    c = np.zeros((n_rows_cost, 1))
    C = np.zeros((n_rows_cost, n_cols))
    C[:,3:] = np.eye(n_rows_cost)
    C[:,:-3] -= np.eye(n_rows_cost)

    H, l = to_quadratic_form(C, c)
    H += np.eye(H.shape[1]) * 1e-6 # Add a small value to the diagonal to make H positive definite


    # Define the equality constraints Cx = d
    n_rows_init_goal = 6
    n_rows_placement = n_grasps + 1
    n_rows_grasp = n_grasps
    n_rows_eq = n_rows_init_goal + n_rows_placement + n_rows_grasp
    C_eq = np.zeros((n_rows_eq, n_cols))

    # Initial and goal positions
    C_eq[0:3, 0:3] = np.eye(3)
    C_eq[3:6, -3:] = np.eye(3)

    # Placement constraints
    for i in range(n_rows_placement):
        j = 6*i
        row = 6 + i
        C_eq[row, j] = -1
        C_eq[row, j+3] = 1

    # Grasp constraints
    for i in range(n_rows_grasp):
        row = 6 + n_rows_placement + i
        j = 6*(i+1) - 1
        C_eq[row, j-2] = 1
        C_eq[row, j-1] = -1
        C_eq[row, j+1] = -1
        C_eq[row, j+2] = 1

    d_eq = np.vstack([x_init.reshape(3,1), x_goal.reshape(3,1), np.zeros((n_rows_placement + n_rows_grasp,1))]).flatten()


    # Define the inequality constraints Gx <= h
    n_rows_ineq = A_p.shape[0]*n_points + A_g.shape[0]*(n_points-2) + 2*n_cols
    G = np.zeros((n_rows_ineq, n_cols))
    for i in range(n_points):
        G[i*A_p.shape[0]:(i+1)*A_p.shape[0], 3*i:3*(i+1)] = A_p  # Placement hull
    for i in range(n_points-2):
        row = n_points*A_p.shape[0] + i*A_g.shape[0]
        G[row:row+A_g.shape[0], 3*(i+1):3*(i+2)] = A_g  # Grasp hull

    # Joint limits (inequality constraints)
    row_joint_upper = n_points*A_p.shape[0] + (n_points-2)*A_g.shape[0]
    row_joint_lower = row_joint_upper + n_cols
    G[row_joint_upper:row_joint_lower,:] = np.eye(n_cols)
    G[row_joint_lower:,:] = -np.eye(n_cols)

    h = np.hstack([b_p]*n_points + [b_g]*(n_points-2) + [upper_joint_limits]*n_points + [np.multiply(-1, lower_joint_limits)]*n_points)


    # Solve the QP
    try:
        x = quadprog_solve_qp(H, l, G, h, C_eq, d_eq)
    except:
        return None

    sol = x.reshape(-1, 3)

    return sol


n_grasps = 2
sol = solve_for_n_graps(n_grasps, x_init, x_goal, A_p, b_p, A_g, b_g, lower_joint_limits, upper_joint_limits)
print(sol)


[[-3.14000000e+00  1.57000000e+00 -5.40000000e-02]
 [-3.14000000e+00 -1.57000000e+00 -2.50000000e-02]
 [-2.64233080e-13  1.57000000e+00 -2.50000000e-02]
 [-2.65121258e-13 -1.57000000e+00 -2.50000000e-02]
 [ 3.14000000e+00  1.57000000e+00 -2.50000000e-02]
 [ 3.14000000e+00 -1.57000000e+00 -5.40000000e-02]]


In [10]:
# Set initial configuration
plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.025
)
plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
    plant_context, 0.025
)
diagram.ForcedPublish(diagram_context)

In [11]:
from math import ceil
from time import sleep

def distance(q1,q2):    
    '''Return the euclidian distance between two configurations'''
    return np.linalg.norm(q2-q1)

def lerp(q0,q1,t):    
    return q0 * (1 - t) + q1 * t


def displayedge(q0,q1,vel=2.): #vel in sec.    
    '''Display the path obtained by linear interpolation of q0 to q1 at constant velocity vel'''
    dist = distance(q0,q1)
    duration = dist / vel    
    nframes = ceil(48. * duration)
    f = 1./48.

    for i in range(nframes + 1):
        q = lerp(q0,q1,float(i)/nframes)
        plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(plant_context, q[2])
        plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(plant_context, -q[2])
        plant.GetJointByName("base_revolute_joint", gripper).set_angle(plant_context, q[1])
        plant.GetJointByName("cap_to_base", cap).set_angle(plant_context, -q[0])
        diagram.ForcedPublish(diagram_context)
        sleep(f)

    sleep(f*30)
    
def displaypath(path, verbose=True):
    for q0, q1 in zip(path[:-1],path[1:]):
        if verbose:
            print("From\t[{:.2f}, {:.2f}, {:.2f}]\tto\t[{:.2f}, {:.2f}, {:.2f}]".format(q0[0], q0[1], q0[2], q1[0], q1[1], q1[2]))
        displayedge(q0,q1)
        
path = sol
    
displaypath(path)

From	[-3.14, 1.57, -0.05]	to	[-3.14, -1.57, -0.03]
From	[-3.14, -1.57, -0.03]	to	[-0.00, 1.57, -0.03]
From	[-0.00, 1.57, -0.03]	to	[-0.00, -1.57, -0.03]
From	[-0.00, -1.57, -0.03]	to	[3.14, 1.57, -0.03]
From	[3.14, 1.57, -0.03]	to	[3.14, -1.57, -0.05]


In [12]:
x_init = np.array([-3.14, -0.1, -0.054])
x_goal = np.array([3.14, -0.4, -0.054])

# Define the 8 corner points of the convex hull
x_bounds = [-np.pi, np.pi] # Bottle cap
y_bounds = [-0.5, 0] # Gripper wrist
z_bounds = [-0.055, -0.025] # Gripper finger

lower_joint_limits = np.array([x_bounds[0], y_bounds[0], z_bounds[0]])
upper_joint_limits = np.array([x_bounds[1], y_bounds[1], z_bounds[1]])

z_bounds_grasp = [-0.025, -0.024]

# Generate all corner points
placement_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds])
grasp_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds_grasp])

# Compute the convex hull
placement_hull = ConvexHull(placement_points)
grasp_hull = ConvexHull(grasp_points)

# Get the A and b representation of the half-spaces
A_p, b_p = get_halfspaces(placement_hull)
A_g, b_g = get_halfspaces(grasp_hull)
assert((A_p @ x_init <= b_p).all())
assert((A_p @ x_goal <= b_p).all())


for i in range(20):
    path = solve_for_n_graps(i, x_init, x_goal, A_p, b_p, A_g, b_g, lower_joint_limits, upper_joint_limits)
    if path is not None:
        print("Solution for", i, "grasps")
        print(path)
        displaypath(path)
        break
    print("No solution for", i, "grasps")


No solution for 0 grasps
No solution for 1 grasps
No solution for 2 grasps
No solution for 3 grasps
No solution for 4 grasps
No solution for 5 grasps
No solution for 6 grasps
No solution for 7 grasps
No solution for 8 grasps
No solution for 9 grasps
No solution for 10 grasps
No solution for 11 grasps
No solution for 12 grasps
Solution for 13 grasps
[[-3.14000000e+00 -1.00000000e-01 -5.40000000e-02]
 [-3.14000000e+00 -5.00000000e-01 -2.50000000e-02]
 [-2.65691625e+00 -1.69162477e-02 -2.50000000e-02]
 [-2.65691625e+00 -4.98580139e-01 -2.50000000e-02]
 [-2.17383541e+00 -1.54993010e-02 -2.50000000e-02]
 [-2.17383541e+00 -4.97163706e-01 -2.50000000e-02]
 [-1.69075700e+00 -1.40852985e-02 -2.50000000e-02]
 [-1.69075700e+00 -4.95750215e-01 -2.50000000e-02]
 [-1.20768054e+00 -1.26737530e-02 -2.50000000e-02]
 [-1.20768054e+00 -4.94339178e-01 -2.50000000e-02]
 [-7.24605540e-01 -1.12641771e-02 -2.50000000e-02]
 [-7.24605540e-01 -4.92930107e-01 -2.50000000e-02]
 [-2.41531516e-01 -9.85608352e-03 -2.

In [13]:
import cvxpy as cp
import os
os.environ["MOSEKLM_LICENSE_FILE"] = "mosek.lic"

def construct_path_from_theta_alpha(theta, alpha):
    path = []
    open_finger = -0.054
    closed_finger = -0.025
    path.append([alpha[0], theta[0], open_finger])
    for i in range(1, len(theta) - 2, 2):
        path.append([alpha[i], theta[i], closed_finger])
        path.append([alpha[i+1], theta[i+1], closed_finger])
    path.append([alpha[-1], theta[-1], open_finger])
        
    return np.array(path)

# Given parameters
theta_min = 0  # Min wrist rotation (deg)
theta_max = 1.5   # Max wrist rotation (deg)
alpha_start = -3.1  # Initial cap angle (deg)
alpha_goal = 3.1  # Goal cap angle (deg)
theta_start= 0.5
max_steps = 12  # Upper bound on steps #TODO: Check: Needs to be even


# Decision variables
alpha = cp.Variable(max_steps)  # Rotation of the cap at each step
theta = cp.Variable(max_steps)  # Wrist angle at each step=

# Constraints
constraints = []

# Initial and final conditions
constraints.append(alpha[0] == alpha_start)  
constraints.append(theta[0] == theta_min)
constraints.append(alpha[max_steps - 1] == alpha_goal)
constraints.append(alpha[max_steps - 1] == alpha[max_steps - 2])

for i in range(max_steps - 1):
    if i%2 == 1:
        constraints.append(alpha[i+1] <= alpha[i] + (theta_max - theta[i]))  # Cap rotation constraint
        constraints.append(theta[i+1] == theta[i] + (alpha[i+1] - alpha[i]))  # TODO: This is the step I did not properly understand
    else:
        constraints.append(alpha[i+1] == alpha[i])
    constraints.append(theta_min <= theta[i])  # Wrist limit constraints
    constraints.append(theta[i] <= theta_max)
    constraints.append(alpha[i] <= alpha_goal)  # Cap limit constraints
    constraints.append(alpha[i+1] >= alpha[i])


# Define objective function
# objective = cp.Minimize(cp.sum_squares(theta[1:] - theta[:-1]))  # Example
objective = cp.Minimize(cp.sum([2**i * (alpha[i+1] - alpha[i])**2 for i in range(max_steps - 1)]))

# Define and solve the problem using MOSEK
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK, verbose=False)  # Use MOSEK as the solver

print("Optimal solution:", theta.value, alpha.value)


Optimal solution: [-0.00000000e+00 -2.35589546e-09  1.50000001e+00 -2.13037052e-09
  1.50000000e+00 -1.90463514e-09  1.50000000e+00  7.00016941e-02
  1.42999830e+00  4.91043909e-01  8.31047294e-01  0.00000000e+00] [-3.1        -3.1        -1.59999999 -1.59999999 -0.09999999 -0.09999999
  1.40000001  1.40000001  2.75999661  2.75999661  3.1         3.1       ]


/usr/local/lib/python3.10/dist-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/usr/local/lib/python3.10/dist-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/usr/local/lib/python3.10/dist-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


In [14]:

displaypath(construct_path_from_theta_alpha(theta.value, alpha.value))

From	[-3.10, -0.00, -0.05]	to	[-3.10, -0.00, -0.03]
From	[-3.10, -0.00, -0.03]	to	[-1.60, 1.50, -0.03]
From	[-1.60, 1.50, -0.03]	to	[-1.60, -0.00, -0.03]
From	[-1.60, -0.00, -0.03]	to	[-0.10, 1.50, -0.03]
From	[-0.10, 1.50, -0.03]	to	[-0.10, -0.00, -0.03]
From	[-0.10, -0.00, -0.03]	to	[1.40, 1.50, -0.03]
From	[1.40, 1.50, -0.03]	to	[1.40, 0.07, -0.03]
From	[1.40, 0.07, -0.03]	to	[2.76, 1.43, -0.03]
From	[2.76, 1.43, -0.03]	to	[2.76, 0.49, -0.03]
From	[2.76, 0.49, -0.03]	to	[3.10, 0.83, -0.03]
From	[3.10, 0.83, -0.03]	to	[3.10, 0.00, -0.05]


In [15]:

def solve_using_cvxpy(n_grasps, x_init, x_goal, lower_joint_limits, upper_joint_limits):
    # Given parameters
    theta_min = lower_joint_limits[1]  # Min wrist rotation (rad)
    theta_max = upper_joint_limits[1]  # Max wrist rotation (rad)
    alpha_start = x_init[0]  # Initial cap angle (rad)
    alpha_goal = x_goal[0]  # Goal cap angle (rad)
    theta_start= x_init[1]
    max_steps = 2*n_grasps + 2  # Upper bound on steps


    # Decision variables
    alpha = cp.Variable(max_steps)  # Rotation of the cap at each step
    theta = cp.Variable(max_steps)  # Wrist angle at each step=

    # Constraints
    constraints = []

    # Initial and final conditions
    constraints.append(alpha[0] == alpha_start)  
    constraints.append(theta[0] == theta_min)
    constraints.append(alpha[max_steps - 1] == alpha_goal)
    constraints.append(alpha[max_steps - 1] == alpha[max_steps - 2])

    for i in range(max_steps - 1):
        if i%2 == 1:
            constraints.append(alpha[i+1] <= alpha[i] + (theta_max - theta[i]))  # Cap rotation constraint
            constraints.append(theta[i+1] == theta[i] + (alpha[i+1] - alpha[i]))  # TODO: This is the step I did not properly understand
        else:
            constraints.append(alpha[i+1] == alpha[i])
        constraints.append(theta_min <= theta[i])  # Wrist limit constraints
        constraints.append(theta[i] <= theta_max)
        constraints.append(alpha[i] <= alpha_goal)  # Cap limit constraints
        constraints.append(alpha[i+1] >= alpha[i])


    # Define objective function
    # objective = cp.Minimize(cp.sum_squares(theta[1:] - theta[:-1]))  # Example
    objective = cp.Minimize(cp.sum([i**2 * (alpha[i+1] - alpha[i])**2 for i in range(max_steps - 1)]))
    #TODO: 2**i returns an error for more than 8 grapss

    # Define and solve the problem using MOSEK
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)  # Use MOSEK as the solver
    
    if prob.status == cp.INFEASIBLE:
        return None
    return construct_path_from_theta_alpha(theta.value, alpha.value)
    

# Comparing times of the two solvers

In [16]:
x_init = np.array([-3.14, -0.1, -0.054])
x_goal = np.array([3.14, -0.4, -0.054])

# Define the 8 corner points of the convex hull
x_bounds = [-np.pi, np.pi] # Bottle cap
y_bounds = [-0.5, 0] # Gripper wrist
z_bounds = [-0.055, -0.025] # Gripper finger

lower_joint_limits = np.array([x_bounds[0], y_bounds[0], z_bounds[0]])
upper_joint_limits = np.array([x_bounds[1], y_bounds[1], z_bounds[1]])

z_bounds_grasp = [-0.025, -0.024]

# Generate all corner points
placement_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds])
grasp_points = np.array([[x, y, z] for x in x_bounds for y in y_bounds for z in z_bounds_grasp])

# Compute the convex hull
placement_hull = ConvexHull(placement_points)
grasp_hull = ConvexHull(grasp_points)

# Get the A and b representation of the half-spaces
A_p, b_p = get_halfspaces(placement_hull)
A_g, b_g = get_halfspaces(grasp_hull)

import time
num_iters = 20

print("solving using cvxpy")
start_cvxpy = time.time()
for i in range(num_iters):
    path = solve_using_cvxpy(i, x_init, x_goal, lower_joint_limits, upper_joint_limits)
    if path is not None:
        stop_cvxpy = time.time()
        print("Solution for", i, "grasps")
        print("CVXPY time:", stop_cvxpy - start_cvxpy)
        print(path)
        displaypath(path, verbose=False)
        break
    # print("No solution for", i, "grasps")

print("solving using quadprog")
start_qp = time.time()
for i in range(num_iters):
    path = solve_for_n_graps(20, x_init, x_goal, A_p, b_p, A_g, b_g, lower_joint_limits, upper_joint_limits)
    if path is not None:
        stop_qp = time.time()
        print("Solution for", 20, "grasps")
        print("Quadprog time:", stop_qp - start_qp)
        print(path)
        displaypath(path, verbose=False)
        break
    # print("No solution for", i, "grasps")

solving using cvxpy
Solution for 13 grasps
CVXPY time: 2.2314178943634033
[[-3.14000000e+00 -5.00000000e-01 -5.40000000e-02]
 [-3.14000000e+00 -5.00000003e-01 -2.50000000e-02]
 [-2.63999996e+00  3.53749061e-08 -2.50000000e-02]
 [-2.63999996e+00 -5.00000002e-01 -2.50000000e-02]
 [-2.13999995e+00  6.92354662e-09 -2.50000000e-02]
 [-2.13999995e+00 -5.00000002e-01 -2.50000000e-02]
 [-1.63999994e+00  6.73610856e-09 -2.50000000e-02]
 [-1.63999994e+00 -5.00000002e-01 -2.50000000e-02]
 [-1.13999993e+00  6.72834255e-09 -2.50000000e-02]
 [-1.13999993e+00 -5.00000002e-01 -2.50000000e-02]
 [-6.39999926e-01  6.58299948e-09 -2.50000000e-02]
 [-6.39999926e-01 -5.00000002e-01 -2.50000000e-02]
 [-1.39999919e-01  6.13111928e-09 -2.50000000e-02]
 [-1.39999919e-01 -5.00000001e-01 -2.50000000e-02]
 [ 3.60000088e-01  5.60940053e-09 -2.50000000e-02]
 [ 3.60000088e-01 -5.00000000e-01 -2.50000000e-02]
 [ 8.60000093e-01  4.71320455e-09 -2.50000000e-02]
 [ 8.60000093e-01 -4.99999998e-01 -2.50000000e-02]
 [ 1.360